In [ ]:
import json
import re
import os
import requests
import time
import itertools
from tqdm import tqdm
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from concurrent.futures import ThreadPoolExecutor
session = HTMLSession()

In [ ]:
def write_to_json(lst, fn, type_=None):
    import json

    with open(fn, "a") as file:
        for x in lst:
            if type_ == "json":
                x = json.dumps(x, ensure_ascii=False)
            else:
                x = str(x)

            file.write(x + "\n")

# get model ids

In [ ]:
max_workers = 20
vehicle = "cars"
category = "best"

def get_model_link(page):
    url = f"https://www.zigwheels.my/{category}-{vehicle}?businessUnit={vehicle}&page={page}"

    r = session.get(url)
    soup = BeautifulSoup(r.content, "lxml")

    models_div = soup.find_all(
        "div", class_="splide__list", attrs={"data-model-url": True}
    )
    model_ids = [int(div.get("data-modelid")) for div in models_div]
    write_to_json(model_ids, f"{os.getcwd()}/output/zigwheels-{category}-{vehicle}-model-ids.txt")


url = f"https://www.zigwheels.my/{category}-{vehicle}?businessUnit={vehicle}&page=1"
r = session.get(url)
soup = BeautifulSoup(r.content, "lxml")
page_count = soup.find("input", {"name": "pageCount"}).get("value")

pages = list(range(1, int(page_count) + 1))
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(get_model_link, x) for x in pages]

    for future in tqdm(futures, total=len(pages)):
        result = future.result()

# get image

In [ ]:
import os
import requests

with open(f"{os.getcwd()}/output/zigwheels-{category}-{vehicle}-model-ids.txt", "r") as file:
    values = file.read().splitlines()
val = ",".join(values)


r = requests.get(
    f"https://www.zigwheels.my/listing/get-images?businessUnit={vehicle}&modelIds={val}&imageType=2"
)
for key, value in r.json().items():
    for val in value:
        data = {"img_url": val["url"], "img_title": val["title"], "model_id": key}
        write_to_json(
            [data], f"{os.getcwd()}/output/zigwheels-{category}-{vehicle}-img.json", type_="json"
        )